# Clasificador de imágenes

In [7]:
import os
import shutil
import random

os.makedirs("data/train/cat", exist_ok=True)
os.makedirs("data/train/dog", exist_ok=True)
os.makedirs("data/test/cat", exist_ok=True)
os.makedirs("data/test/dog", exist_ok=True)

all_images = os.listdir("train")

random.shuffle(all_images)

for img in all_images[:10000]:  # 5000 por clase para train
    label = img.split(".")[0]
    shutil.copy(f"train/{img}", f"data/train/{label}/{img}")

for img in all_images[10000:12000]:  # 1000 por clase para test
    label = img.split(".")[0]
    shutil.copy(f"train/{img}", f"data/test/{label}/{img}")


FileNotFoundError: [Errno 2] No such file or directory: 'train'

## Procesado de imágenes y visualización

In [ ]:
import matplotlib.pyplot as plt
import os
from tensorflow.keras.preprocessing.image import load_img, img_to_array

def mostrar_imagenes(clase, path="data/train", n=9):
    files = os.listdir(f"{path}/{clase}")[:n]
    plt.figure(figsize=(10, 10))
    for i, file in enumerate(files):
        img = load_img(f"{path}/{clase}/{file}", target_size=(200, 200))
        plt.subplot(3, 3, i+1)
        plt.imshow(img)
        plt.axis("off")
    plt.suptitle(f"Imágenes de {clase}")
    plt.show()

mostrar_imagenes("cat")
mostrar_imagenes("dog")


<Figure size 1000x1000 with 0 Axes>

<Figure size 1000x1000 with 0 Axes>

## Entreno de la red neuronal

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_data = train_datagen.flow_from_directory(
    "data/train",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)

test_data = test_datagen.flow_from_directory(
    "data/test",
    target_size=(224, 224),
    batch_size=32,
    class_mode='categorical'
)


Found 0 images belonging to 2 classes.
Found 0 images belonging to 2 classes.


In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(2, activation='softmax'))  # 2 clases: gato y perro

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])



2025-05-20 20:27:20.373998: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-05-20 20:27:20.407723: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-20 20:27:20.689233: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-20 20:27:20.810054: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1747772841.112965   12862 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1747772841.20

## Entreno del modelo

In [ ]:
history = model.fit(
    train_data,
    validation_data=test_data,
    epochs=5  # Puedes usar más si tienes tiempo
)


NameError: name 'train_data' is not defined

## Optimización

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

checkpoint = ModelCheckpoint("best_model.keras", monitor='val_loss', save_best_only=True)
earlystop = EarlyStopping(monitor='val_loss', patience=3)

model.fit(
    train_data,
    validation_data=test_data,
    epochs=10,
    callbacks=[checkpoint, earlystop]
)


## Guardado del modelo

In [ ]:
model.save("final_model.keras")
